In [ ]:
import os
import gc
import warnings
from warnings import simplefilter

import numpy as np
import pandas as pd


warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

IS_OFFLINE = False
IS_INFER = True
TRAINING = True
MAX_LOOKBACK = np.nan
SPLIT_DAY = 435

In [ ]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
df_shape = df.shape

In [ ]:
def reduce_mem_usage(df, verbose=0):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
               
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")
    return df

In [ ]:
from numba import njit, prange
from itertools import combinations


@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"imb3_{a}_{b}_{c}" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features

In [ ]:
from sklearn.decomposition import PCA


def imbalance_features(df):

    df["bid_ask_spread"] = df["ask_price"] - df["bid_price"]
    df["bid_ask_spread_diff"] = df.groupby(["stock_id"])["bid_ask_spread"].diff()  # change in bid ask spread by stock

    df["volume"] = df.eval("bid_size + ask_size")
    df["total_bid_value"] = df.eval("bid_size * bid_price")
    df["total_ask_value"] = df.eval("ask_size * ask_price")

    df["log_return"] = df["wap"].apply(lambda x: np.log(x) if x is not None else x)

    df['imb_s1'] = df.eval("(bid_size - ask_size) / (bid_size + ask_size)")  # liquidity imbalance
    df['imb_s2'] = df.eval("(imbalance_size - matched_size) / (matched_size + imbalance_size)")  # matched imbalance
    df["imbalance_ratio"] = df.eval("imbalance_size / matched_size")

    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["imbalance_size", "bid_size", "ask_size", "matched_size"]

    for c in combinations(prices, 2):
        df[f"imb2_{c[0]}_{c[1]}"] = df.eval(f"({c[0]} - {c[1]}) / ({c[0]} + {c[1]})")

    for c in [prices, sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values

    # Statistical aggregation features
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"sizes_{func}"] = df[sizes].agg(func, axis=1)

    return df.replace([np.inf, -np.inf], 0)


def other_features(df):
    df["day_of_week"] = df["date_id"] % 5

    pca = PCA(n_components=1)
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    df["pca_prices"] = pca.fit_transform(df[prices].fillna(1))

    return df


def feat_eng(df):
    df = imbalance_features(df)
    df = other_features(df)
    gc.collect()  
    features = [feat for feat in df.columns if feat not in ["row_id", "time_id", "date_id", "target"]]
    return df[features]

In [ ]:
if IS_OFFLINE:
    df_train = df[df["date_id"] <= SPLIT_DAY]
    df_valid = df[df["date_id"] > SPLIT_DAY]
    print("[OFFLINE]")
    print(f"train: {df_train.shape}\nvalid: {df_valid.shape}")
else:
    df_train = df
    print("[ONLINE]")

In [ ]:
if TRAINING:
    if IS_OFFLINE:
        X_train = feat_eng(df_train)
        print("Training set feature engeering finished.")
        X_valid = feat_eng(df_valid)
        print("Valid set feature engeering finished.")
        X_valid = reduce_mem_usage(X_valid)
    else:
        X_train = feat_eng(df_train)
        print("[ONLINE] Training set feature engeering finished.")
X_train = reduce_mem_usage(X_train)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import joblib  # for saving and loading models


lgb_params = {
    "objective": "mae",
    "n_estimators": 6000,
    "num_leaves": 256,
    "subsample": 0.6,
    "colsample_bytree": 0.8,
    "learning_rate": 0.00871,
    'max_depth': 11,
    "n_jobs": 4,
    "device": "gpu",
    "verbosity": -1,
    "importance_type": "gain",
}

features = X_train.columns.to_list()
print(f"Number of features: {len(features)}")

num_folds = 5
fold_size = 480 // num_folds
gap = 5

models = []
scores = []

model_save_path = 'models' 
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

date_ids = X_train['date_id'].values

for i in range(num_folds):
    start = i * fold_size
    end = start + fold_size
    if i < num_folds - 1:  # no need to purge after the last fold
        purged_start = end - 2
        purged_end = end + gap + 2
        train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
    else:
        train_indices = (date_ids >= start) & (date_ids < end)
    test_indices = (date_ids >= end) & (date_ids < end + fold_size)
    
    X_train_fold = X_train[train_indices]
    y_train_fold = df_train['target'][train_indices]
    X_valid_fold = X_train[test_indices]
    y_valid_fold = df_train['target'][test_indices]

    print(f"Fold {i+1} LightGBM training")
    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_train_fold[features],
        y_train_fold,
        eval_set=[(X_valid_fold[features], y_valid_fold)],
        callbacks=[
            lgb.callback.early_stopping(stopping_rounds=100),
            lgb.callback.log_evaluation(period=100),
        ],
    )

    models.append(model)
    filename = os.path.join(model_save_path, f'fold_{i+1}.txt')
    model.booster_.save_model(filename)
    print(f"Fold {i+1} model saved.")

    # Evaluate model performance on the validation set
    y_pred = model.predict(X_valid_fold[features])
    score = mean_absolute_error(y_pred, y_valid_fold)
    scores.append(score)
    print(f"Fold {i+1} MAE: {score}")

    # Free up memory by deleting fold specific variables
    del X_train_fold, y_train_fold, X_valid_fold, y_valid_fold
    gc.collect()

# Calculate the average best iteration from all folds
average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

# Update the lgb_params with the average best iteration
lgb_params['n_estimators'] = average_best_iteration

print(f"Training final model with average best iteration: {average_best_iteration}")

# Train the final model on the entire dataset
model = lgb.LGBMRegressor(**lgb_params)
model.fit(
    X_train[features],
    df_train['target'],
    callbacks=[
        lgb.callback.log_evaluation(period=100),
    ],
)

models.append(model)
filename = os.path.join(model_save_path, 'lgbm.txt')
model.booster_.save_model(filename)
print(f"Final LightGBM model saved.")

print(f"Average MAE across all folds: {np.mean(scores)}")

In [ ]:
# feature importance
feat_imp = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(feat_imp)

In [ ]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

In [ ]:
if IS_INFER:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0

    y_min, y_max = -64, 64
    cache = pd.DataFrame()
    model_weights = [1/len(models)] * len(models)
    for (test, revealed_targets, sample_prediction) in iter_test:
        cache = pd.concat([cache, test], ignore_index=True, axis=0)
        if counter > 0:
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        feature = feat_eng(cache)

        lgb_predictions = np.zeros(len(test))
        for model, weight in zip(models, model_weights):
            lgb_predictions += weight * model.predict(feature)
        lgb_predictions = zero_sum(lgb_predictions, test['bid_size'] + test['ask_size'])

        clipped_predictions = np.clip(lgb_predictions, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)

        counter += 1